In [ ]:
import numpy as np

def get_transient_timestamps_mod(
    neural_data, thresh_type="zscore", std_thresh=5
):
    """
    Converts an array of continuous time series (e.g., traces or S)
    into lists of timestamps where activity exceeds some threshold.

    :parameters
    ---
    neural_data: (neuron, time) array
        Neural time series, (e.g., C or S).

    thresh_type: str
        Type of thresholding ("zscore" or "zscore_localMax").

    std_thresh: float
        Number of standard deviations above the mean to define threshold.

    :returns
    ---
    event_times: list of length neuron
        Each entry in the list contains the timestamps of a neuron's
        activity.

    event_mags: list of length neuron
        Event magnitudes.

    bool_arr: ndarray of bool
        Boolean array indicating whether a value exceeds the threshold.
    """
    # Compute thresholds for each neuron.
    neural_data = np.asarray(neural_data, dtype=np.float32)
    stds = np.std(neural_data, axis=1)
    means = np.mean(neural_data, axis=1)
    thresh = means + std_thresh * stds

    # Get event times, magnitudes, and boolean array.
    event_times = []
    event_mags = []
    bool_arr = np.zeros_like(neural_data, dtype=bool)

    for index, (neuron, t) in enumerate(zip(neural_data, thresh)):
        event_indices = []
        
        if thresh_type == "zscore":
            event_indices = np.where(neuron > t)[0]
        
        elif thresh_type == "zscore_localMax":
            for i in range(1, len(neuron) - 1):
                if neuron[i] > t and neuron[i] > neuron[i - 1] and neuron[i] > neuron[i + 1]:
                    event_indices.append(i)
        
        event_times.append(np.array(event_indices))
        event_mags.append(neuron[event_indices])
        bool_arr[index, event_indices] = True

    return event_times, event_mags, bool_arr

In [ ]:
# Need to add option to separate this into left and right

import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd

def get_behavior_onsets(behavior_data, num_frames):
    onsets = np.zeros(num_frames)
    for _, row in behavior_data.iterrows():
        onsets[row['scopeFrameStart']:row['scopeFrameEnd'] + 1] = 1

    return onsets

def calculate_auroc_with_shuffling(matrix, behavior_df, behavior_names, num_shuffles=1000, cleversysFlag=0):
    num_neurons = matrix.shape[0]
    num_behaviors = len(behavior_names)
    auroc_matrix = np.zeros((num_neurons, num_behaviors))
    p_value_matrix = np.zeros((num_neurons, num_behaviors))

    for i in range(num_neurons):
        neuron_data = matrix[i, :]
        print(i)

        for j, behavior_name in enumerate(behavior_names):
            if cleversysFlag==0:
                behavior_data = behavior_df[behavior_df['Behavior'] == behavior_name]
            else:
                behavior_data = behavior_df[behavior_df['EventType'] == behavior_name]

            if behavior_data.empty:
                print(f"No data found for behavior '{behavior_name}' for neuron {i + 1}.")
                continue

            behavior_onsets = get_behavior_onsets(behavior_data, matrix.shape[1])

            true_auroc = roc_auc_score(behavior_onsets, neuron_data)

            # Shuffle scopeStartTime and scopeEndTime
            shuffled_aurocs = []
            for _ in range(num_shuffles):
                shuffled_behavior_data = behavior_data.copy()
                shuffled_behavior_data['scopeFrameStart'] = np.random.randint(0, matrix.shape[1] - 1, len(behavior_data))
                shuffled_behavior_data['scopeFrameEnd'] = shuffled_behavior_data['scopeFrameStart'] + (behavior_data['scopeFrameEnd'] - behavior_data['scopeFrameStart']).values
                behavior_onsets_shuffled = get_behavior_onsets(shuffled_behavior_data, matrix.shape[1])
                #print(behavior_onsets_shuffled)
                shuffled_aurocs.append(roc_auc_score(behavior_onsets_shuffled, neuron_data))

            auroc_matrix[i, j] = true_auroc
            p_value_matrix[i, j] = (np.sum(shuffled_aurocs >= true_auroc) + 1) / (num_shuffles + 1)

    return auroc_matrix, p_value_matrix

In [ ]:
auROC_matrix, p_value_matrix = calculate_auroc_with_shuffling(C_arrays[0], boris1s[0], ['noncontact investigation'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'calcium_matrix' is your calcium data as a numpy array (neurons x frames)

def mean_and_median(calcium_matrix):
    # Calculate mean, median, and standard deviation
    mean_activity = np.mean(calcium_matrix, axis=1)
    median_activity = np.median(calcium_matrix, axis=1)
    std_deviation = np.std(calcium_matrix, axis=1)

    # Display statistics for each neuron
    for neuron_id in range(mean_activity.shape[0]):
        print(f"Neuron {neuron_id + 1}:")
        print(f"Mean Activity: {mean_activity[neuron_id]:.2f}")
        print(f"Median Activity: {median_activity[neuron_id]:.2f}")
        print(f"Standard Deviation: {std_deviation[neuron_id]:.2f}")
        print()

    # Create bar plots to visualize statistics
    neuron_ids = np.arange(mean_activity.shape[0])
    plt.bar(neuron_ids, mean_activity, label='Mean', alpha=0.7)
    plt.bar(neuron_ids, median_activity, label='Median', alpha=0.7)
    plt.xlabel('Neuron')
    plt.ylabel('Activity')
    plt.title('Mean and Median Activity')
    plt.legend()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming 'calcium_matrix' is your calcium data as a numpy array (neurons x frames)


# Calculate mean, median, and standard deviation
mean_activity = np.mean(calcium_matrix, axis=1)
median_activity = np.median(calcium_matrix, axis=1)
std_deviation = np.std(calcium_matrix, axis=1)

# Display statistics using print statements
for neuron_id, (mean, median, std) in enumerate(zip(mean_activity, median_activity, std_deviation)):
    print(f"Neuron {neuron_id + 1} - Mean: {mean:.2f}, Median: {median:.2f}, Std Dev: {std:.2f}")

# Create a scatter plot of mean vs. standard deviation
plt.scatter(mean_activity, std_deviation, c='blue', marker='o', label='Neurons')
plt.xlabel('Mean Activity')
plt.ylabel('Standard Deviation')
plt.title('Mean vs. Standard Deviation of Neural Activity')
plt.legend()
plt.grid()
plt.show()





In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_behavior_time_pie(behavior_df):
    # Calculate the minimum and maximum frame values
    min_frame = behavior_df['scopeFrameStart'].min()
    max_frame = behavior_df['scopeFrameEnd'].max()
    
    # Initialize a dictionary to store behavior durations
    behavior_durations = {}
    
    # Calculate durations for each behavior
    unique_behaviors = behavior_df['Behavior'].unique()
    for behavior_name in unique_behaviors:
        behavior_data = behavior_df[behavior_df['Behavior'] == behavior_name]
        behavior_durations[behavior_name] = np.sum(behavior_data['scopeFrameEnd'] - behavior_data['scopeFrameStart'])
    
    # Calculate the duration of frames with no behavior
    total_duration = max_frame - min_frame
    for behavior_name in unique_behaviors:
        total_duration -= behavior_durations.get(behavior_name, 0)
    
    # Add the 'No Behavior' category
    behavior_durations['No Behavior'] = total_duration
    
    # Create the pie chart
    plt.figure(figsize=(8, 8))
    plt.pie(behavior_durations.values(), labels=behavior_durations.keys(), autopct='%1.1f%%', startangle=140, fontsize=8)
    plt.title('Total Time Spent in Each Behavior')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.show()

# Example usage
# Assuming behavior_df is your behavior dataframe
plot_behavior_time_pie(behavior_df)